# Assignment 1: Dino Fun World

You, in your role as a burgeoning data explorer and visualizer, have been asked by the administrators of a small amusement park in your hometown to answer a couple questions about their park operations. In order to perform the requested analysis, they have provided you with a database containing information about one day of the park's operations.

### Provided Database

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkins', 'attractions', and 'sequences'. The information contained in each of these tables is listed below:

`checkins`:
    - Description: check-in data for all visitors for the day in the park. The data includes two types of check-ins, inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type
`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carussel, etc.
    - Fields: AttractionID, Name, Region, Category, type
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
The database is named 'dinofunworld.db' and is located in the 'readonly' directory of the Jupyter Notebook environment. It can be accessed at 'readonly/dinofunworld.db'.

### Questions to Answer

The administrators would like you to answer four relatively simple questions about the park activities on the day in question. These questions all deal with park operations and can be answered using the data provided.

Question 1: What is the most popular attraction to visit in the park?
Question 2: What ride (note that not all attractions are rides) has the longest visit time?
Question 3: Which Fast Food offering has the fewest visitors?
Question 4: Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline.

#### Administrative Notes

This assignment will be graded by Coursera's grading system. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly. Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed.

In [1]:
import sqlite3
from datetime import timedelta
import operator

db_filename = 'dinofunworld.db'
conn = sqlite3.connect(db_filename)
c = conn.cursor()

# Graded Cell
# PartID:  NDnou
# Question 1: What is the most popular attraction to visit in the park?
# Notes: Your output should be the name of the attraction.

c.execute("select name from checkin, attraction where checkin.attraction = attraction.attractionid group by name order by count(*) desc limit 1")
most_popular_attraction = c.fetchone()[0]
#print("1. Most popular attraction to visit in the park is:")
print(most_popular_attraction)

Atmosfear


In [2]:
# Graded Cell
# PartID: FXGHp
# Question 2: What ride (note that not all attractions are rides) has the longest average visit time?
# Notes: Your output should be the name of the ride.

durations = {}
c.execute("Select attractionid from attraction where lower(category) like lower('%ride%')")
all_attractionids = c.fetchall()

for attractionid in all_attractionids:
    rideid = attractionid[0]
    c.execute("select duration from checkin where attraction=" + str(rideid))
    duration = 0
    checkin_durations = c.fetchall()
    
    for i in checkin_durations:
        try:
            time = timedelta(int(i[0].split(':')[0]), int(i[0].split(':')[1]), int(i[0].split(':')[2])).total_seconds()
            duration += time
        except (ValueError, AttributeError) as e:
          #  print(e)
            continue

    durations[rideid] = duration

max_wait_attraction = max(durations.items(), key=operator.itemgetter(1))[0]
c.execute("select Name from attraction where AttractionID=" + str(max_wait_attraction))

#print("2. The ride with the longest average visit time is:")
print(c.fetchone()[0])

Flight of the Swingodon


In [3]:
# Graded Cell
# PartID: KALua
# Question 3: Which Fast Food offering in the park has the fewest visitors?
# Notes: Your output should be the name of the fast food offering.

c.execute("select name from attraction, checkin where attraction.attractionid = checkin.attraction and attraction.type='Fast Food' group by name order by count(*)")
#print("3. The fast food shop with the fewest visitors is:")
print(c.fetchone()[0])

Theresaur Food Stop


In [4]:
# Graded Cell
# PartID: B0LUP
# Question 4: Compute the Skyline of number of visits and visit time for the park's ride and 
#  report the rides that appear in the Skyline. 
# Notes: Remember that in this case, higher visits is better and lower visit times are better. 
#  Your output should be formatted as an array listing the names of the rides in the Skyline.

rides = {}
c.execute("select attractionid from attraction where lower(category) like lower('%ride%')")
all_attractionids = c.fetchall()

for attractionid in all_attractionids:
    rideid = attractionid[0]
    c.execute("select duration from checkin where attraction=" + str(rideid))
    duration_of_ride = 0
    
    for d in c.fetchall():
        try:
            duration_of_ride += timedelta(int(d[0].split(':')[0]), int(d[0].split(':')[1]), int(d[0].split(':')[2])).total_seconds()
        except (ValueError, AttributeError) as e:
            continue
            
    rides[rideid] = [duration_of_ride]
    c.execute("select count(*) from checkin where attraction=" + str(rideid))
    count = c.fetchone()[0]
    rides[rideid].append(count)

ride1 = list(rides.keys())[0]
ride2 = ride1
ride3 = ride2

for ride in rides:
    if rides[ride][0] < rides[ride2][0]:
        ride2 = ride
    if rides[ride][1] > rides[ride1][1]:
        ride1 = ride
    if (rides[ride][0] < rides[ride3][0]) and (rides[ride][1] > rides[ride3][1]):
        ride3 = ride

c.execute("select name from attraction where attractionid in (" + str(ride1) + "," + str(ride2) + "," + str(ride3) + ")")
final_rides = c.fetchall()
for r in final_rides:
    print(r[0])

Wrightiraptor Mountain
Atmosfear
Dykesadactyl Thrill
